In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# I. Loading Raw Stopward dataset

In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/WR_project/StopWord_Full.csv"
df = pd.read_csv(path, encoding="utf-8")

df.head()

,term,linguistic_group
0,ធ្វើតាម,Content word (verb: to follow)
1,ទៅដល់ទីនោះ,Prepositions / Relational Words
2,នាង,Pronouns
3,ឆ្លៀតសម័យ,Function Nouns
4,ផ្តល់នូវ,Auxiliary Verbs / Aspect Markers


In [ ]:
df.columns


Index(['term', 'linguistic_group'], dtype='object')

In [ ]:
df = df.drop(columns=["linguistic_group"])
df.head()

,term
0,ធ្វើតាម
1,ទៅដល់ទីនោះ
2,នាង
3,ឆ្លៀតសម័យ
4,ផ្តល់នូវ


# II. Handle Duplicated words

In [ ]:
# Check if duplicates exist in the stopword column (first column)
duplicates = df[df.duplicated(subset=df.columns[0], keep=False)]
print("Duplicate stopwords:")
print(duplicates)

Duplicate stopwords:
                      term
38                      ទេ
43                   ផ្ទុយ
44                      ជា
137             ផ្តល់អំណោយ
160                 សំរាប់
170              យ៉ាងណាមិញ
217                     ក៏
218                     ក៏
234                  ច្រើន
235                 មិនបាន
240                    បាន
245                    មិន
255                    មិន
259                     ទេ
260               ពេលណាមិញ
262                ត្រូវតែ
296                  ច្រើន
301                 សំរាប់
306                 ម្នាក់
308                   ទាំង
311               ការរៀបចំ
314                   យ៉ាង
320                   យ៉ាង
322                     ជា
325               ពេលណាមិញ
326                    មិន
328                    ឡើង
342                    ដូច
344               ការរៀបចំ
347              យ៉ាងណាមិញ
368                 មិនបាន
393                    មិន
422                  នេះជា
438                ត្រូវតែ
439                    បាន
498    

In [ ]:
# Keep only unique stopwords
df_unique = df.drop_duplicates(subset=df.columns[0])

# Convert to set for fast lookup
stopwords = set(df_unique.iloc[:, 0].astype(str).str.strip())

print("Total unique stopwords:", len(stopwords))
print(list(stopwords)[:20])

Total unique stopwords: 726
['ប្រាកដ', 'ដល់កន្លែង', 'ពេលណានោះហើយ', 'ផ្តល់ភាពសំរាប់', 'ផ្តល់អនុសាសន៍', 'គឺ', 'ប៉ុន្តែ', 'យើងទាំងអស់', 'ទៅដល់ទីនោះ', 'ប៉ុន្នឹងក៏បាន', 'មុខ', 'ផ្តល់បញ្ជូនគ្នា', 'បន្ថែម', 'ផ្តល់តួនាទី', 'ប្រគល់ទៅ', 'នេះក៏អាច', 'ហាក់បីដូចម្តេច', 'ធ្លាក់ចុះ', 'ផ្តល់ការរាប់បញ្ចូល', 'ប្រហែលជា']


# III.Loading Clean text dataset

In [ ]:
Khmertext_segmented = "//content/drive/MyDrive/WR_project/khmertext_segmented.txt"
Khmer_text = []
with open(Khmertext_segmented, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:  # ignore empty lines
            Khmer_text.append(line)

print("Total lines loaded:", len(Khmer_text))
print(Khmer_text[:5])


Total lines loaded: 130807
['សេចក្តីថ្លែងការណ៍ ពាក់ព័ន្ធ នឹង ព័ត៌មាន នេះ   បាន ឲ្យដឹង ថា   លោក   នឹង ទៅ បំពេញ ទស្សនកិច្ច នៅ ប្រទេស ឥណ្ឌា   តាម ការអញ្ជើញ របស់ នាយក រដ្ឋមន្ត្រី ឥណ្ឌា   លោក   ដោយ ចាត់ទុក ការធ្វើដំណើរ នេះ ថា   មាន សារៈសំខាន់   ដោយ ផ្តល់ ឱកាស ឲ្យ មាន កិច្ចពិភាក្សា គ្រប់ ជ្រុងជ្រោយ   អំពី របៀបវារៈ ទាំងមូល នៃ ទំនាក់ទំនង រុស្ស៊ី ឥណ្ឌា', 'ផ្តន្ទាទោស ដាក់ ពន្ធនាការ ដោយ ព្យួរ ទោស   រឺ មិន ព្យួរ ទោស ក្នុង រយ   ពេល ឆ្នាំ កន្លងទៅ ទេ   នោះ តុលាការ អាច សំរេច មិន ដាក់ទោស   ប៉ុន្តែ ត្រូវ', 'នៅ ឆ្នាំ   ភាគ កណ្ដាល នៃ ចក្រភព បាន បែកបាក់   បង្កើត បានជា រាជាណាចក្រ តូច ឯករាជ្យ ជាច្រើន', 'ក្នុង នោះ រួម មាន   បួរ ហ្គូញ លើ   បួរ ហ្គូញ ត្រង់ ហ្ស៊ូរ៉ាន   និង អ៊ីតាលី', 'ចំណែកឯ តំបន់ ឡូរ៉ែ ន   ត្រូវបាន បញ្ចូល ទៅ ភាគ ខាងកើត   ក្លាយជា រាជាណាចក្រ ចំណុះ']


In [ ]:
def remove_stopwords(text, stopwords):
    words = text.split()  # Khmer text is often space-separated
    cleaned = [w for w in words if w not in stopwords]
    return " ".join(cleaned)


# IV.Apply Stopword in text dataset
In this step, i remove the stopword from my file stopwods in my text dataset that i have cleaned from preprocessing step.

In [ ]:
# Apply to all lines
cleaned_dataset = [remove_stopwords(s, stopwords) for s in Khmer_text]

# Print first 100 fully cleaned sentences
print("First 100 fully cleaned sentences:")
for i, sent in enumerate(cleaned_dataset[:100], 1):
    print(f"{i}: {sent}")


First 100 fully cleaned sentences:
1: សេចក្តីថ្លែងការណ៍ ពាក់ព័ន្ធ ព័ត៌មាន ឲ្យដឹង ថា ទស្សនកិច្ច នៅ ប្រទេស ឥណ្ឌា ការអញ្ជើញ នាយក រដ្ឋមន្ត្រី ឥណ្ឌា ដោយ ចាត់ទុក ការធ្វើដំណើរ ថា សារៈសំខាន់ ដោយ កិច្ចពិភាក្សា ជ្រុងជ្រោយ របៀបវារៈ ទាំងមូល ទំនាក់ទំនង រុស្ស៊ី ឥណ្ឌា
2: ផ្តន្ទាទោស ពន្ធនាការ ដោយ ព្យួរ ទោស រឺ ព្យួរ ទោស រយ ឆ្នាំ កន្លងទៅ តុលាការ ដាក់ទោស ត្រូវ
3: នៅ ឆ្នាំ ភាគ កណ្ដាល ចក្រភព បែកបាក់ បង្កើត បានជា រាជាណាចក្រ តូច ឯករាជ្យ ជាច្រើន
4: បួរ ហ្គូញ បួរ ហ្គូញ ហ្ស៊ូរ៉ាន អ៊ីតាលី
5: តំបន់ ឡូរ៉ែ ន ភាគ ខាងកើត ក្លាយជា រាជាណាចក្រ ចំណុះ
6: ស្តេច រាជាណាចក្រ ថ្មី កាន់អំណាច ការគាំទ្រ ពួក អភិជន តំបន់ ប្រឆាំង ទាមទារ រាជ្យ ខ្សែ រាជវង្ស ការ៉ូឡាំង ហ្ស៊ាំង
7: នៅ ភាគ ខាងកើត ពួក អភិជន តំបន់ ជ្រើសរើស តែងតាំង ពួក ឌុក កាន់អំណាច
8: ស្ថានភាព ផ្លាស់ប្តូរ នៅ ឆ្នាំ នៅ ល្វីស ឡង់ ហ្វង់ ស្តេច ខ្សែ រាជវង្ស ការ៉ូឡាំង ហ្ស៊ាំង ចុងក្រោយ បង្អស់ ហ្វ្រង់ស៊ី ខាងកើត សោយទិវង្គត
9: ការបែកបាក់ បញ្ចៀស ពួក មន្ត្រី ជាន់ ខ្ពស់ រាជាណាចក្រ ជ្រើសរើស កុងរ៉ាដ ទី ឡើងសោយរាជ្យ
10: កុងរ៉ាដ សមាជិក រាជវង្ស ការ៉ូឡាំង ហ្ស៊ាំង ក៏ដោយ ទ្រង់ ជនជាតិ ហ្វ្រង់ ខ្សែ ត